# Website
### https://www.hksuning.com/

***

# Installation
### If you are running this notebook for the first time, please the follow one sell, otherwise, please ignore.

In [0]:
!pip install pyppeteer
!pip install beautifulsoup4
!pip install nest_asyncio
!pip install asyncio

# Importing

In [0]:
# coding=utf-8

from pyppeteer import launch
import asyncio
import nest_asyncio
from bs4 import BeautifulSoup as bs
import re

In [0]:
nest_asyncio.apply()

# Websites that are needed to be crawling

In [0]:
urls = [
        "https://search.hksuning.com/search/list?ci=503692", # url_tablet
        "https://search.hksuning.com/search/list?ci=503694", # url_laptop
        "https://search.hksuning.com/search/list?ci=503304" # url_phone
]

# Return all web pages of each category

In [0]:
async def get_item(categroy: str):
    browser = await launch()
    page = await browser.newPage()
    await page.goto(categroy)
    # pages = await browser.pages()
    # await browser.close()
    return page

# Start crawling

## Get pages of each item page

In [0]:
dict_items = {}
list_pages = []
next_btn_selector = "#bottom_pager > div > a.next"

In [0]:
async def get_page_num(page) -> int:
    content = bs(await page_tablet.content(), 'lxml')
    page_num = re.findall(re.compile(r"共\d頁"), content.prettify())[0][1]
    return int(page_num)

In [0]:
for url in urls:
    page = asyncio.get_event_loop().run_until_complete(get_item(url))
    list_pages.append(page)

In [53]:
page_tablet = asyncio.get_event_loop().run_until_complete(get_item(url_tablet))

page_num_tablet = asyncio.get_event_loop().run_until_complete(
        get_page_num(page_tablet)
    )
print(f"There are total {page_num_tablet} pages about tablet")

There are total 2 pages about tablet


In [54]:
page_laptop = asyncio.get_event_loop().run_until_complete(get_item(url_laptop))

page_num_laptop = asyncio.get_event_loop().run_until_complete(
        get_page_num(page_laptop)
    )
print(f"There are total {page_num_laptop} pages about laptop")

There are total 2 pages about laptop


In [9]:
page_phone = asyncio.get_event_loop().run_until_complete(get_item(url_phone))

page_num_phone = asyncio.get_event_loop().run_until_complete(
        get_page_num(page_phone)
    )
print(f"There are total {page_num_phone} pages about mobile phone")

TimeoutError: ignored

## Next Page

### Have next page button?

In [0]:
async def have_button(page, btn_selector) -> bool:
    
    try:
        await page_tablet.click(btn_selector)
        return True
    except:
        return False

### Click next page button

In [0]:
async def click_next_page(page) -> bool:
    
    next = asyncio.get_event_loop().run_until_complete(
            have_button(page, next_btn_selector)
        )

    if next:
        await page_tablet.click(next_btn_selector)
        return True
    else:
        # print("No next page!")
        return False

## Get all item url

In [0]:
async def get_url(page):
    source = bs(await page.content(), 'lxml').prettify()

    item_id = re.findall(
            re.compile(r'(?<=//product.hksuning.com/0000000000/)(\d*).html'),
            source
        )
    item_id = list(dict.fromkeys(item_id))
    tmp_dict = {}
    
    for i in item_id:
        url_patten = f"https://product.hksuning.com/0000000000/{i}.html"
        tmp_dict[i] = url_patten
        dict_items.update(tmp_dict)

# asyncio.get_event_loop().run_until_complete(get_url(page_tablet))

## Crawl data

In [74]:
async def get_data(page):
    while asyncio.get_event_loop().run_until_complete(
            have_button(page, next_btn_selector)
        ):
        # Get URLs
        asyncio.get_event_loop().run_until_complete(get_url(page))
        # Click next page button
        asyncio.get_event_loop().run_until_complete(get_url(page))

    # asyncio.get_event_loop().run_until_complete(click_next_page(page))

asyncio.get_event_loop().run_until_complete(get_data(page_tablet))
# len(dict_items)

80

# Demo codes

In [0]:
async def demo():
    browser = await launch()
    page_tablet = await browser.newPage()
    await page_tablet.goto(url_phone)
    
    doc = bs(await page_tablet.content(), 'lxml')
    doc.prettify()
    print(doc.title)
    await browser.close()

asyncio.get_event_loop().run_until_complete(demo())

TimeoutError: ignored